This notebook provides functions to plot the results

The function plot_mean_std plots the mean and standard deviations for each method, and compares it to the true values and the calibration measures. 

The function dist2 computes the RMSRE between prediction y1 and true values y2.

The function compare_errors gathers the RMSRE and levels of prediction intervals of all method, and compute $p^{0.9}_{N,M}$.

The function plot_errors plots the RMSRE and $p^{0.9}_{N,M}$ for each method.

In [1]:
variable_names = [r"$\ell_{F} - \ell_{0}$", r"$r_{F}$", r"$\epsilon_{max}$"] #names of the three outputs


def plot_mean_std(index_calib, pre_path,savefig = False):
    
    list_values = [true_values, results_measures]
    list_sigma = [[0]*3, sigma]
    list_labels = ["True value", "Measure"]

    incr = 0.09 #distance between the error bars
    plot_hierarchical_plugin = pd.read_csv(pre_path + f"hierarchical_model/calib_{index_calib}/plot_alpha_map_lamdba_bayesian.csv", index_col=0) #get mean and std for hierarchical plugin
    plot_full_bayes = pd.read_csv(pre_path + f"hierarchical_model/calib_{index_calib}/full_bayes.csv", index_col=0) # get mean and std for hierarchical full bayes
    plot_no_error = pd.read_csv(pre_path + f"no_error/calib_{index_calib}/plot_alpha_map_lamdba_bayesian.csv", index_col=0) # get mean and std for no error
    plot_unif_error = pd.read_csv(pre_path + f"uniform_error/calib_{index_calib}/plot_alpha_map_lamdba_bayesian.csv", index_col=0) #get mean and std for uniform error
    elinewidth = 3 #error bar width
    markersize = 8 
    Ysimu_embed = pd.read_csv(pre_path + f"embedded_discrepancy/calib_{index_calib}/predictions.csv", index_col=0) #get mean for embedded discrepancy
    Ystd_embed = pd.read_csv(pre_path + f"embedded_discrepancy/calib_{index_calib}/std_dev.csv", index_col=0) #get std for embedded discrepancy
    
    x = np.arange(len(results_measures))
    ticks = [f"$x_{{{k+1}}}$" for k in x]

    fig, axes = plt.subplots(3, 1, figsize=(30, 13))  # 3 subplots 
    
    for i, ax in enumerate(axes, start=1):
        if index_calib == i: #if the output is the one observed
            ax.errorbar(x, (list_values[1][f"Y{i}"]-list_values[0][f"Y{i}"])/list_values[0][f"Y{i}"], yerr=list_sigma[1][i-1]/list_values[0][f"Y{i}"], fmt='o', color='blue', label=list_labels[1],elinewidth=elinewidth, markersize = markersize) #plot measures and variance noise
        
        ax.scatter(x, list_values[0][f"Y{i}"]-list_values[0][f"Y{i}"], marker='x', color='blue', label=list_labels[0], s=120,linewidths=4) #plot true values
        ax.errorbar(x + incr, (plot_no_error.iloc[:10, i-1]-list_values[0][f"Y{i}"])/list_values[0][f"Y{i}"], yerr=plot_no_error.iloc[10:, i-1]/list_values[0][f"Y{i}"], fmt='o', color='green', label='No error',elinewidth=elinewidth, markersize = markersize)  #plot no_error
        ax.errorbar(x + 2*incr, (plot_unif_error.iloc[:10, i-1]-list_values[0][f"Y{i}"])/list_values[0][f"Y{i}"], yerr=plot_unif_error.iloc[10:, i-1]/list_values[0][f"Y{i}"], fmt='o', color='red', label='Uniform error',elinewidth=elinewidth,markersize = markersize) #plot uniform error
        ax.errorbar(x + 3*incr, (plot_hierarchical_plugin.iloc[:10, i-1]-list_values[0][f"Y{i}"])/list_values[0][f"Y{i}"], yerr=plot_hierarchical_plugin.iloc[10:, i-1]/list_values[0][f"Y{i}"], fmt='o', color='purple', label="Hierarchical \n     MAP",elinewidth=elinewidth,markersize = markersize) #plot hierarchical map
        ax.errorbar(x + 4*incr, (plot_full_bayes.iloc[:10, i-1]-list_values[0][f"Y{i}"])/list_values[0][f"Y{i}"], yerr=plot_full_bayes.iloc[10:, i-1]/list_values[0][f"Y{i}"], fmt='o', color='magenta', label="Hierarchical \n full Bayes",elinewidth=elinewidth,markersize = markersize) #plot full bayes
        ax.errorbar(x + 5*incr, (Ysimu_embed.iloc[:, i-1]-list_values[0][f"Y{i}"])/list_values[0][f"Y{i}"], yerr=Ystd_embed.iloc[:, i-1]/list_values[0][f"Y{i}"], fmt='o', color='orange', label="Embedded \ndiscrepancy",elinewidth=elinewidth,markersize = markersize) #plot embedded discrepancy
        ax.axhline(y=0, color='gray', linestyle='--', linewidth=0.8)

        ax.set_title(f"Prediction of {variable_names[i-1]} from measures of {variable_names[index_calib-1]}", fontsize=42)
        
        if i == len(axes): #x ticks on the last subplots
            ax.set_xticks(x)
            ax.set_xticklabels(ticks, fontsize=30)
        else:
            ax.set_xticks([])  # Remove x-ticks for the first two subplots
        
        ax.tick_params(axis='y', labelsize=20)
    
    handles, labels = axes[-1].get_legend_handles_labels()
    fig.legend(handles, labels, loc='center right', fontsize=30, bbox_to_anchor=(1.15, 0.5))

    plt.tight_layout()
    if savefig: plt.savefig(pre_path + f"plots/plot_pred_{index_calib}.pdf",bbox_inches='tight')
    plt.show()
    
def dist2(y1,y2):
    return np.sqrt(np.mean((y1-y2)**2/y2**2))

def compare_errors(index_calib, pre_path):
    errors_MAP = pd.read_csv(pre_path + f"hierarchical_model/calib_{index_calib}/errors_map.csv", index_col = 0).values.transpose() #get error hierarchical MAP
    intervals_MAP = pd.read_csv(pre_path + f"hierarchical_model/calib_{index_calib}/intervals_map.csv", index_col = 0).values #get interval levels hierarchical MAP


    errors_bayes = pd.read_csv(pre_path + f"hierarchical_model/calib_{index_calib}/errors_bayes.csv", index_col = 0).values.transpose() #get error full bayes
    intervals_bayes = pd.read_csv(pre_path + f"hierarchical_model/calib_{index_calib}/intervals_bayes.csv", index_col = 0).values #get interval levels full bayes

    errors_allp = pd.read_csv(pre_path + f"uniform_error/calib_{index_calib}/errors_map.csv", index_col = 0).values.transpose() #get error uniform error
    intervals_allp = pd.read_csv(pre_path + f"uniform_error/calib_{index_calib}/intervals_map.csv", index_col = 0).values #get interval levels uniform error

    errors_noerror = pd.read_csv(pre_path + f"no_error/calib_{index_calib}/errors_map.csv", index_col = 0).values.transpose() #get error no error
    intervals_noerror = pd.read_csv(pre_path + f"no_error/calib_{index_calib}/intervals_map.csv", index_col = 0).values #get interval levels no error

    errors_embed = pd.read_csv(pre_path + f"embedded_discrepancy/calib_{index_calib}/error_pred.csv", index_col = 0).values.transpose() #get error embedded discrepancy
    intervals_embed = pd.read_csv(pre_path + f"embedded_discrepancy/calib_{index_calib}/interv_errors.csv", index_col = 0).values #get error embedded discrepancy

    res = pd.DataFrame(np.transpose(np.concatenate([errors_noerror,errors_allp, errors_MAP, errors_bayes, errors_embed]))) #gather prediction errors
    columns =[ "No error","  Uniform \n errors"," Hierarchical \n MAP", "  Hierarchical \n full Bayes", " Embedded \n discrepancy"] 
    nb_compare = 5

    res.columns = columns
    res.index = np.array(range(3))+1
    res_intervals = pd.DataFrame(np.transpose(np.concatenate([[np.apply_along_axis(lambda x: sorted(x)[-2], 0, intervals_noerror)],[np.apply_along_axis(lambda x: sorted(x)[-2], 0, intervals_allp)],[np.apply_along_axis(lambda x: sorted(x)[-2], 0, intervals_MAP)],[np.apply_along_axis(lambda x: sorted(x)[-2], 0, intervals_bayes)], [np.apply_along_axis(lambda x: sorted(x)[-2], 0, intervals_embed)]]))) #gather 90% quantiles of the interval levels
    res_intervals.columns = columns

    return res, res_intervals


def plot_errors(index_calib, pre_path, savefig = False):
    errors = compare_errors(index_calib=index_calib, pre_path=pre_path) #Get RMSRE and p^0.9_M,N
    errors = errors[0] * 100, errors[1] * 100 #convert to %
    names = errors[0].columns
    x = np.arange(len(names))
    bar_width1 = 23 / 100
    bar_width2 = 18 / 100
    alph = 0.7
    fonttext = 35
    loc_bar = [-2 / 3 * bar_width1, 2 / 3 * bar_width2]
    loc_bar = loc_bar + [loc_bar[1] + bar_width2, loc_bar[1] + 2 * bar_width2]

    fig, axes = plt.subplots(3, 1, figsize=(37, 13), sharex=True, sharey=True)  # 3 rows, 1 column

    patches = [
        mpatches.Patch(color='#1f77b4', label='RMSRE (%)', alpha=alph),
        mpatches.Patch(color='green', label=r'$\hat{p}^{0.9}$ (%)', alpha=alph),
    ] # legend

    for idx, ax1 in enumerate(axes, start=1):
        ax1.bar(x + loc_bar[0], errors[0].iloc[idx-1, :].values, width=bar_width1, alpha=alph) #bars for RMSRE
        
        ax2 = ax1.twinx() #twin y-axis

        ax2.bar(x + loc_bar[1], errors[1].iloc[idx-1, :].values, color="green", width=bar_width1, alpha=alph) #bars for p^0.9

        for kk in range(len(x)):
            ax1.text(x[kk] + loc_bar[0], errors[0].iloc[idx-1, kk] * 0.95, "{:.1f}".format(errors[0].iloc[idx-1, kk]), ha='center', va="top", fontsize=fonttext, fontweight='bold') #text value of rmsre
            ax2.text(x[kk] + loc_bar[1], errors[1].iloc[idx-1, kk] * 0.95, str(round(errors[1].iloc[idx-1, kk])), ha='center', va="top", fontsize=fonttext, fontweight='bold') #text value of p^0.9

        ax1.tick_params(axis='x', labelsize=30)
        ax1.tick_params(axis='y', labelsize=20)
        ax2.tick_params(axis='y', labelsize=20)

        ax1.locator_params(axis='y', nbins=3)
        ax2.locator_params(axis='y', nbins=3)

        ax1.set_xticks(x)
        ax1.set_xticklabels(names)

        ax1.set_title(f"Prediction error of {variable_names[idx-1]} from measures of {variable_names[index_calib-1]}", fontsize=42)

    fig.legend(handles=patches, loc='center right', fontsize=40, bbox_to_anchor=(1.1, 0.5))

    fig.text(0.04, 0.5, 'RMSRE (%)', va='center', rotation='vertical', fontsize=38)
    fig.text(0.94, 0.5, 'Prediction interval level (%)', va='center', rotation='vertical', fontsize=38)

    plt.tight_layout(rect=[0.05, 0, 0.93, 1])  # Adjust layout to make room for y labels

    if savefig: plt.savefig(pre_path + f"plots/plot_err_{index_calib}.pdf",bbox_inches='tight')

    
    plt.show()

